In [1]:
import numpy as np
import sys
import time
import memory_profiler
import syft as sy
from syft.workers.websocket_client import WebsocketClientWorker
import torch
from torch.utils.data import Dataset, DataLoader
#from torchvision import datasets, transforms
#from syft.frameworks.torch.federated import utils

import run_websocket_client_xiot as rwc
import logging

args = rwc.define_and_get_arguments(args=[])
use_cuda = args.cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
print(args)

Namespace(batch_size=128, cuda=False, epochs=2, federate_after_n_batches=50, lr=0.01, save_model=False, seed=1, test_batch_size=1000, use_virtual=False, verbose=False)


In [2]:
hook = sy.TorchHook(torch)

kwargs_websocket = {"host": "localhost", "hook": hook, "verbose": args.verbose}
alice = WebsocketClientWorker(id="alice", port=8777, **kwargs_websocket)
bob = WebsocketClientWorker(id="bob", port=8778, **kwargs_websocket)
charlie = WebsocketClientWorker(id="charlie", port=8779, **kwargs_websocket)
jane = WebsocketClientWorker(id="jane", port=8780, **kwargs_websocket)

workers = [alice, bob, charlie, jane]
print(workers)

[<WebsocketClientWorker id:alice #tensors local:0 #tensors remote: 0>, <WebsocketClientWorker id:bob #tensors local:0 #tensors remote: 0>, <WebsocketClientWorker id:charlie #tensors local:0 #tensors remote: 0>, <WebsocketClientWorker id:jane #tensors local:0 #tensors remote: 0>]


In [3]:
#run this box only if the the next box gives pipeline error
#Get data set

class IoTDataset(Dataset):
    # Initialize your data, download, etc.

    def __init__(self):
        
        benign = np.loadtxt("benign_traffic.csv", delimiter = ",")
        mirai = np.loadtxt("mirai_traffic.csv", delimiter = ",")
        gafgyt = np.loadtxt("gafgyt_traffic.csv", delimiter = ",")
        alldata = np.concatenate((benign, gafgyt, mirai))
        j = len(benign[0])
        data = alldata[:, 1:j] 
        benlabel = alldata[:, 0]
        bendata = (data - data.min()) / (data.max() - data.min())
        self.len = alldata.shape[0]
        self.x_data = torch.from_numpy(bendata)
        self.y_data = torch.from_numpy(benlabel)

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index]

    def __len__(self):

        return self.len

full_dataset = IoTDataset()

train_size = int(len(full_dataset)* 0.8)
test_size = len(full_dataset) - train_size

# split the dataset
trainset, testset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
trainset = trainset.dataset
testset = testset.dataset


federated_train_loader = sy.FederatedDataLoader(
            trainset.federate(tuple(workers)),
            batch_size= args.batch_size,
            shuffle=True,
            iter_per_worker=True
    )

test_loader = DataLoader(
           dataset=testset,  batch_size=args.batch_size, shuffle=True)

        
model = rwc.nmodel().to(device)
print(model)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter("%(asctime)s %(levelname)s %(filename)s(l:%(lineno)d) - %(message)s")
handler.setFormatter(formatter)
logger.handlers = [handler]

for epoch in range(1, args.epochs + 1):
    print("Starting epoch {}/{}".format(epoch, args.epochs))
    #starttbase = time.time()
    #startmbase = memory_profiler.memory_usage()
    starttefi = time.time()
    startmefi = memory_profiler.memory_usage()
    model = rwc.train(model, device, federated_train_loader, args.lr, args.federate_after_n_batches)
    #endtbase =time.time()
    #endmbase = memory_profiler.memory_usage()
    #traintime_base = endtbase - starttbase
    #train_memory_base = endmbase[0] - startmbase[0]
    #print("Training time base: {:2f} sec".format(traintime_base))
    #print("Training memory base: {:2f} mb".format(train_memory_base))
    #rwc.test(model, device, test_loader)
    endtefi = time.time()
    endmefi = memory_profiler.memory_usage()
    traintime_efi = endtefi - starttefi
    train_memory_efi = endmefi[0] - startmefi[0]
    print("Training time optimize: {:2f} sec".format(traintime_efi))
    print("Training memory optimize: {:2f} mb".format(train_memory_efi))
    rwc.test(model, device, test_loader)
    

Sequential(
  (0): Linear(in_features=115, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=128, bias=True)
  (7): ReLU()
  (8): Linear(in_features=128, out_features=1, bias=True)
  (9): Sigmoid()
)
Starting epoch 1/2


2022-01-30 18:34:49,215 DEBUG run_websocket_client_xiot.py(l:262) - Starting training round, batches [0, 50]
2022-01-30 18:34:49,363 WARNING websocket_client.py(l:107) - Websocket connection closed (worker: alice)
2022-01-30 18:34:49,516 WARNING websocket_client.py(l:112) - Created new websocket connection


WebSocketConnectionClosedException: Connection is already closed.